In [1]:
import torch
from data_handler import create_dataset, save_json, load_json, load_encoded_data
from model import TravelDocClassifier
from cuda_utils import clear_cache, print_cuda_memory_info

c:\Users\wei\Desktop\programs\information-retrieval\ir_final_doc_classifier\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clear_cache()
print_cuda_memory_info()

Device: NVIDIA GeForce RTX 4070 SUPER
Total Memory: 12.88 GB
Reserved Memory: 0.00 GB
Allocated Memory: 0.00 GB
Free Memory: 0.00 GB


In [3]:
encoded_data = load_encoded_data('encoded_data/all_512')
filtered_data = load_json('filtered_data/all.json')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = TravelDocClassifier(device=device, params_path='parameters/20240603_95_model_parameters.pth')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
test_dataset = create_dataset(encoded_data)

pred_labels = classifier.predict_all(test_dataset)

travel_related = []
non_travel_related = []

for i, pred_label in enumerate(pred_labels):
    if pred_label == 1:
        travel_related.append(filtered_data[i])
    else:
        non_travel_related.append(filtered_data[i])

len(travel_related), len(non_travel_related)

c:\Users\wei\Desktop\programs\information-retrieval\ir_final_doc_classifier\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


(35246, 18265)

In [7]:
save_json(travel_related, 'results/travel_related_predictions.json')
save_json(non_travel_related, 'results/non_travel_related_predictions.json')